# Workflow for Classification

## 0. Working on colab
You have to clone the repo and get some credentials in order to work on colab and access gpu compautation

In [ ]:
!git clone https://github.com/MorelElian/Hackaton_Quantum;

## 1. Processing data

In [ ]:
from Hackaton_Quantum.data_processing.process import create_label_folder,generate_train,generate_test

In [ ]:
## You might want to change those paths, depending on the fact that you are working on collab or not
DATA_PATH = './Hackaton_Quantum/data/ai_ready/ai_ready/images'
CSV_PATH = './Hackaton_Quantum/data/ai_ready/ai_ready/x-ai_data.csv'
FINAL_PATH =  './Hackaton_Quantum/data/ai_ready/ai_ready/traitement/'
VGG_MODEL_PATH = './Hackaton_Quantum/models/stored_models/VGG/'
INCEPTION_MODEL_PATH = './Hackaton_Quantum/models/stored_models/INCEPTION/'
create_label_folder(    
    DATA_PATH,
    CSV_PATH,
    FINAL_PATH
)
train, validation = generate_train(
    FINAL_PATH + 'train/',
    FINAL_PATH + 'validation/'
)

## 2. Modeling

The folders models/stored_models/VGG and models/stored_models/INCEPTION contains pre trained models for this tasks, if you want to load them -> load_models = true.  
If you want to calculate new weights, set load_models to false : it can be much longer

In [ ]:
from Hackaton_Quantum.models.classification_models import generate_inception,generate_VGG
load_models = True

In [ ]:
vgg_model = generate_VGG() # generate VGG returns a fitted model
inception_model = generate_inception()
if load_models : 
    vgg_model.load_weights(VGG_MODEL_PATH)
    inception_model.weights(INCEPTION_MODEL_PATH)
else:
    vgg_model.fit_generator(train, validation_data = validation, steps_per_epoch = 50, epochs = 10)
    inception_model.fit_generator(train, validation_data = validation, steps_per_epoch = 50, epochs = 10)

In [ ]:
save_models = False

In [ ]:
if save_models:
    vgg_model.save_weights(VGG_MODEL_PATH)
    inception_model.save_weights(INCEPTION_MODEL_PATH)


## 3. Visualising performance

In [ ]:
from Hackaton_Quantum.vizualisation.vizu import prompt_roc_curve

silos_for_validation, pas_silos_for_validation = generate_test(FINAL_PATH + '/validation')

prompt_roc_curve(vgg_model,
                inception_model,
                silos_for_validation,
                pas_silos_for_validation,
                pas = 15
                )

We can now select the best threshold depending on waht we want to do.   
For example, in order to do the segmentation , we need a very good precision, therefore, we will consider the vgg model with a threshold equal to 0.7

## 4. Prediction on test/validation data

In [ ]:
from Hackaton_Quantum.evaluation.predict import evaluate,predict

In [ ]:
# We first need to transform the image into tensors which can be exploited by our model, it will then make predictions step by step for every new image

    
image_to_predict = generate_test(
    FINAL_PATH + '/test'
)
# We recompute the metrics for the best models selected
metrics_evaluation_inception = evaluate(inception_model,silos_for_validation,pas_silos_for_validation,threshold = 0.7)
metrics_evaluation_vgg = evaluate(vgg_model,silos_for_validation,pas_silos_for_validation, threshold = 0.6)

predictions_on_test_inception = predict(inception_model,image_to_predict)
predictions_on_test_vgg = predict(vgg_model,image_to_predict)

## 